<a href="https://colab.research.google.com/github/YKhalitov/College-Projects/blob/main/RIT-Projects/Machine%20Learning%20-%20CSCI%20335/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:



print ("hi")

hi
